# Wheel vs PMCC vs LEAPS vs Buy and hold

## Load historical IV

In [1]:
import math
import datetime
import yfinance as yf

ticker = yf.Ticker('^VIX')
vix_df = ticker.history(period='15y', interval="1d")

## Helper functions

In [2]:
from scipy.stats import norm

def call_price(sp, st, r, t, vol, d):
    d1 = (math.log(sp/st) + (r - d + ((vol**2)/2))*t)/(vol*math.sqrt(t))
    d2 = d1 - vol*math.sqrt(t)
    return sp*math.exp(-d*t)*norm.cdf(d1) - st*math.exp(-r*t)*norm.cdf(d2)

def put_price(sp, st, r, t, vol, d):
    d1 = (math.log(sp/st) + (r - d + ((vol**2)/2))*t)/(vol*math.sqrt(t))
    d2 = d1 - vol*math.sqrt(t)
    return st*math.exp(-r*t)*norm.cdf(-d2) - sp*math.exp(-d*t)*norm.cdf(-d1)

In [3]:
def get_ticker_data(t):
    ticker = yf.Ticker(t) # GS-PD | FP.PA | MSFT | QQQ | SPY | QLD
    df = ticker.history(period='15y', interval="1d")
    df["vola"] = vix_df["Close"]
    return df[['Close', 'Dividends', 'vola']].reset_index().to_numpy()

In [4]:
def find_itm_strike(price, dte, delta):
    strike_price = math.floor(price)-1
    d = call_price(price+1, strike_price, rate, dte/365,long_iv/100,0) - call_price(price, strike_price, rate, dte/365,long_iv/100,0)
    while d < delta and strike_price > 0:
        strike_price = strike_price - 1
        d = call_price(price+1, strike_price, rate, dte/365,long_iv/100,0) - call_price(price, strike_price, rate, dte/365,long_iv/100,0)
    
    if strike_price <= 0:
        raise ValueError('couldnt find itm strike')

    return strike_price

def find_otm_strike(price, dte, delta):
    strike_price = math.floor(price)+1
    d = call_price(price+1, strike_price, rate, dte/365,short_iv/100,0) - call_price(price, strike_price, rate, dte/365,short_iv/100,0)
    while d > delta and strike_price < (price*2):
        strike_price = strike_price + 1
        d = call_price(price+1, strike_price, rate, dte/365,short_iv/100,0) - call_price(price, strike_price, rate, dte/365,short_iv/100,0)
    
    if strike_price <= 0:
        raise ValueError('couldnt find leaps')

    return strike_price

# Global params

In [6]:
daily_investment = 500/30 # how much to invest every monthly divided by 30 to get daily investment
OPT_CONT_SIZE = 100
init_cap = 9000
rate = 0.007
ticker = 'SPY'

# Wheel strategy


In [73]:

# Strategy params
call_opt_percent_strike = 0.03 # % abose/below current price
put_opt_percent_strike = 0.04 # % abose/below current price
opt_duration = 22 # 30 days - 22 trading days
opt_duration_reel = 30 # 30 days - 22 trading days
commissions_per_trade = 0
divi_tax_rate = 0.3 # dividend tax
gain_tax_rate = 0 # 0.3 # capital gains tax



In [74]:
print('starting with: {}'.format(init_cap))

starting with: 9000


In [75]:

def wheel(ticker, call_opt_percent_strike, put_opt_percent_strike, opt_duration, opt_duration_reel):
    series_data = get_ticker_data(ticker)

    # Account
    capital = init_cap
    puts = 0
    calls = 0
    cash_earnings = 0
    invested = init_cap
    shares = 0
    balance = 0
    free_capital = init_cap

    strike = 0
    call_exp_date = None
    put_exp_date = None

    for idx, data in enumerate(series_data):
        date, price, divi, vola = data[0].date(), data[1], data[2], data[3]

        iv = vola
        f_divi = 0

        # Monthly investment (spread daily)
        invested = invested + daily_investment

        capital = capital + divi*shares*(1-divi_tax_rate) + daily_investment
        cash_earnings = cash_earnings + divi*shares*(1-divi_tax_rate)

        free_capital = capital - (-puts*strike*OPT_CONT_SIZE)

        share_buying_power = math.floor(free_capital/price)
        if(share_buying_power >= 1):
            shares = shares + share_buying_power
            capital = capital - share_buying_power*price
            free_capital = capital - (-puts*strike*OPT_CONT_SIZE)


        if((shares >= OPT_CONT_SIZE and calls == 0) or (calls !=0 and isinstance(call_exp_date, datetime.date) and call_exp_date <= date)):
            put_exp_date = None
            # sell covered call
            strike = price*(1+call_opt_percent_strike)
            strike = (math.ceil(strike)+math.floor(strike))/2

            # get futures dividend
            f_divi = 0
            for i in range(idx+1, min(idx+opt_duration+1, len(series_data))):
                if(series_data[i][2] != 0):
                    f_divi = series_data[i][2]

            calls = -math.floor(shares/OPT_CONT_SIZE)

            prem = call_price(price, strike, rate, opt_duration_reel/365,iv/100,f_divi/price)*OPT_CONT_SIZE*abs(calls)*(1-gain_tax_rate)
            capital = capital + prem - commissions_per_trade*abs(puts)
            cash_earnings = cash_earnings + prem - commissions_per_trade*abs(puts)

            call_exp_date = date + datetime.timedelta(days=opt_duration_reel)

            print('sell covered call')
            print('date: {}'.format(date))
            print('calls: {}'.format(calls))
            print('price: {}'.format(price))
            print('strike: {}'.format(strike))
            print('premium: {}'.format(prem))
            print('call_exp_date: {}'.format(call_exp_date))
            print('vola: {}'.format(vola))
            print('shares: {}'.format(shares))
            print('capital: {}'.format(capital))
            print('free_capital: {}'.format(free_capital))
            print('balance: {}'.format(balance))
            print('-----')

        if ((shares < 100 and puts == 0 and math.floor(capital/(price*OPT_CONT_SIZE)) >= 1) or (puts !=0 and isinstance(put_exp_date, datetime.date) and put_exp_date <= date)):
            call_exp_date = None
            # sell cash covered put
            strike = price*(1-put_opt_percent_strike)
            strike = (math.ceil(strike)+math.floor(strike))/2

            puts = -max(math.floor(capital/(strike*OPT_CONT_SIZE)), 1)

            # get futures dividend
            f_divi = 0
            for i in range(idx+1, min(idx+opt_duration+1, len(series_data))):
                if(series_data[i][2] != 0):
                    f_divi = series_data[i][2]
    
            prem = put_price(price, strike, rate, opt_duration_reel/365,iv/100,f_divi/price)*OPT_CONT_SIZE * abs(puts)*(1-gain_tax_rate)
            capital = capital + prem - commissions_per_trade*abs(puts)
            cash_earnings = cash_earnings + prem - commissions_per_trade*abs(puts)

            put_exp_date = date + datetime.timedelta(days=opt_duration_reel)

            print('sell cash covered put')
            print('date: {}'.format(date))
            print('puts: {}'.format(puts))
            print('price: {}'.format(price))
            print('strike: {}'.format(strike))
            print('premium: {}'.format(prem))
            print('put_exp_date: {}'.format(put_exp_date))
            print('vola: {}'.format(vola))
            print('shares: {}'.format(shares))
            print('capital: {}'.format(capital))
            print('free_capital: {}'.format(free_capital))
            print('balance: {}'.format(balance))
            print('-----')

        if (calls != 0 and price > strike):
            # Shares get called
            print('Shares get called')
            print('date: {}'.format(date))
            print('price: {}'.format(price))
            print('strike: {}'.format(strike))

            shares_in_contracts = -calls*OPT_CONT_SIZE
            capital = capital + shares_in_contracts*strike
            shares = shares - shares_in_contracts
            calls = 0
            call_exp_date = None
            print('capital: {}'.format(capital))
            print('free_capital: {}'.format(free_capital))
            print('balance: {}'.format(balance))
            print('-----')

        if (puts != 0 and price < strike):
            # Get assigned shares
            print('Get assigned shares')
            print('date: {}'.format(date))
            print('price: {}'.format(price))
            print('strike: {}'.format(strike))
            
            shares_in_contracts = -puts*OPT_CONT_SIZE
            capital = capital - shares_in_contracts*strike
            shares = shares + shares_in_contracts
            puts = 0
            put_exp_date = None
            print('capital: {}'.format(capital))
            print('free_capital: {}'.format(free_capital))
            print('balance: {}'.format(balance))
            print('-----')
        balance = capital + shares*price
    return balance, invested, len(series_data)

In [76]:
balance, invested, days = wheel(ticker, call_opt_percent_strike, put_opt_percent_strike, opt_duration, opt_duration_reel)

sell covered call
date: 2006-12-06
calls: -1
price: 105.16661834716797
strike: 108.5
premium: 31.03893651061931
call_exp_date: 2007-01-05
vola: 11.329999923706055
shares: 100
capital: 35.35716872904332
free_capital: 4.318232218424015
balance: 10513.123336791992
-----
sell covered call
date: 2007-01-05
calls: -1
price: 104.82763671875
strike: 107.5
premium: 52.16853404509649
call_exp_date: 2007-02-04
vola: 12.140000343322754
shares: 103
capital: 142.84773280750576
free_capital: 90.67919876240927
balance: 10958.07499394877
-----
Shares get called
date: 2007-02-01
price: 107.86344909667969
strike: 107.5
capital: 10766.95841243397
free_capital: 16.958412433970647
balance: 11473.04086075754
-----
sell covered call
date: 2007-02-02
calls: -1
price: 108.01261901855469
strike: 111.5
premium: 22.642349204217638
call_exp_date: 2007-03-04
vola: 10.079999923706055
shares: 106
capital: 113.01814546794044
free_capital: 90.3757962637228
balance: 11522.002556110729
-----
sell covered call
date: 2007-0

In [77]:
returns = (balance/invested) - 1
annual_returns = math.pow((1+returns), 365/days) - 1

print('returns: {}%'.format(round(returns*100,2)))
print('annual returns: {}%'.format(round(annual_returns*100,2)))

returns: 1825.33%
annual returns: 33.09%


# Buy hold strategy (benchmark)

In [37]:
def hodl(ticker):
    series_data = get_ticker_data(ticker)

    capital = 0
    hld_balance = 0
    hld_cash_earnings = 0
    shares = math.floor(init_cap/series_data[0][1])
    invested = init_cap

    for idx, data in enumerate(series_data):
        date, price, divi, vola = data[0].date(), data[1], data[2], data[3]
        
        # Monthly investment (spread daily)
        invested = invested + daily_investment
        
        capital = capital + (divi*shares*(1-divi_tax_rate)) + daily_investment
        hld_cash_earnings = hld_cash_earnings + (divi*shares*(1-divi_tax_rate))

        share_buying_power = math.floor(capital/price)
        if(share_buying_power >= 1):
            shares = shares + share_buying_power
            capital = capital - share_buying_power*price

    hld_balance = capital + shares*price

    return hld_balance, invested, len(series_data)

In [38]:
hld_balance, invested, days = hodl(ticker)

In [39]:
hld_returns = (hld_balance/invested)-1
hld_annual_returns = math.pow((1+hld_returns), 365/days) - 1

print('hld_returns: {}%'.format(round(hld_returns*100,2)))
print('hld_annual_returns: {}%'.format(round(hld_annual_returns*100,2)))

hld_returns: 289.25%
hld_annual_returns: 14.04%


# Poor Man's Covered Call strategy

### Global params

In [88]:
long_iv = 26
short_iv = 17
mid_iv = 20

leaps_dte = 420
leaps_delta = 0.7
leaps_dte_limit = 60

short_dte = 16
short_delta = 0.25
short_dte_limit = 4
short_profit_limit = 0.5 # 1-short_profit_limit == %P
short_loss_limit = 1.9 # 1-short_loss_limit == %L

In [89]:
def pmcc(ticker,leaps_dte,leaps_delta,leaps_dte_limit,short_dte,short_delta,short_dte_limit,short_profit_limit,short_loss_limit):
    series_data = get_ticker_data(ticker)

    free_capital = init_cap
    invested = init_cap

    leaps_cnt = 0
    leaps_exp_date = None
    leaps_strike = 0

    short_cnt = 0
    short_exp_date = None
    short_strike = 0

    for idx, data in enumerate(series_data):
        date, price, divi, vola = data[0].date(), data[1], data[2], data[3]

        mid_iv = vola
        short_iv = (short_dte/25)*vola
        mid_iv = 1.1*vola
        long_iv = mid_iv*1.3

        # Monthly investment (spread daily)
        free_capital = free_capital + daily_investment
        invested = invested + daily_investment

        # BTO leaps
        if leaps_cnt == 0:
            leaps_strike = find_itm_strike(price, leaps_dte, leaps_delta)
            leaps_price = OPT_CONT_SIZE*call_price(price, leaps_strike, rate, leaps_dte/365,long_iv/100,0)
            leaps_cnt = math.floor(free_capital/leaps_price)
            free_capital = free_capital - leaps_cnt*leaps_price
            leaps_exp_date = date + datetime.timedelta(days=leaps_dte)
            print(date)
            print('BTO leaps')
            print(free_capital)

        # STC leaps
        elif (leaps_exp_date - date).days <= leaps_dte_limit or idx == (len(series_data)-1):
            free_capital = free_capital + leaps_cnt*OPT_CONT_SIZE*call_price(price, leaps_strike, rate, (leaps_exp_date - date).days/365,mid_iv/100,0)
            leaps_exp_date = None
            leaps_cnt = 0
            leaps_strike = 0
            print(date)
            print('STC leaps')
            print(free_capital)

        # STO covered calls
        if leaps_cnt > 0 and short_cnt == 0 and free_capital > 0: 
            short_strike = find_otm_strike(price, short_dte, short_delta)
            short_price = OPT_CONT_SIZE*call_price(price, short_strike, rate, short_dte/365,short_iv/100,0)
            short_cnt = leaps_cnt
            free_capital = free_capital + short_cnt*short_price
            short_exp_date = date + datetime.timedelta(days=short_dte)
            print(date)
            print(price)
            print('sell {} CCs @{} for: {}, expiring: {}'.format(short_cnt, short_strike, short_cnt*short_price, short_exp_date))
            print(free_capital)

        if short_cnt > 0:
            curr_short_price = OPT_CONT_SIZE*call_price(price, short_strike, rate, (short_exp_date - date).days/365,short_iv/100,0)
            
            # BTC covered calls
            if ((short_exp_date - date).days <= short_dte_limit) or (idx == len(series_data)-1) or (curr_short_price/short_price <= short_profit_limit) or (curr_short_price/short_price >= short_loss_limit):
                free_capital = free_capital - short_cnt*curr_short_price
                print(date)
                print(price)
                print('buy back {} CCs @{} for: {}'.format(short_cnt, short_strike, short_cnt*curr_short_price))
                short_exp_date = None
                short_cnt = 0
                short_strike = 0
                short_price = 0
                print(free_capital)

    print(free_capital)
    print(invested)
    
    return free_capital, invested, len(series_data)


In [90]:
pmcc_balance, invested, days = pmcc(ticker,leaps_dte,leaps_delta,leaps_dte_limit,short_dte,short_delta,short_dte_limit,short_profit_limit,short_loss_limit)

2006-09-25
BTO leaps
1210.0754098706402
2006-09-25
98.26825714111328
sell 6 CCs @102 for: 3.8293628863475426, expiring: 2006-10-11
1213.9047727569878
2006-09-26
99.08417510986328
buy back 6 CCs @102 for: 9.476864834686527
1221.094574588968
2006-09-27
99.2028579711914
sell 6 CCs @103 for: 2.5522963726817283, expiring: 2006-10-13
1240.3135376283165
2006-10-02
98.71328735351562
buy back 6 CCs @103 for: 0.29159916839165967
1290.021938459925
2006-10-03
98.92100524902344
sell 6 CCs @102 for: 12.4235895607125, expiring: 2006-10-19
1319.1121946873043
2006-10-04
100.0781021118164
buy back 6 CCs @102 for: 50.47184419095245
1285.3070171630186
2006-10-05
100.27099609375
sell 6 CCs @104 for: 4.0982433692204445, expiring: 2006-10-21
1306.0719271989058
2006-10-06
100.14485168457031
buy back 6 CCs @104 for: 1.8439460515822237
1320.8946478139903
2006-10-09
100.20423889160156
sell 6 CCs @104 for: 2.9744797737588202, expiring: 2006-10-25
1340.5357942544158
2006-10-18
101.31688690185547
buy back 6 CCs @10

In [91]:
returns = (pmcc_balance/invested) - 1
annual_returns = math.pow((1+returns), 365/days) - 1

print('returns: {}%'.format(round(returns*100,2)))
print('annual returns: {}%'.format(round(annual_returns*100,2)))

returns: 904.12%
annual returns: 24.97%


# Just LEAPS

In [92]:

def leaps(ticker,leaps_dte,leaps_delta,leaps_dte_limit):
    series_data = get_ticker_data(ticker)

    free_capital = init_cap
    invested = init_cap

    leaps_cnt = 0
    leaps_exp_date = None
    leaps_strike = 0

    for idx, data in enumerate(series_data):
        date, price, divi, vola = data[0].date(), data[1], data[2], data[3]

        # Monthly investment (spread daily)
        free_capital = free_capital + daily_investment
        invested = invested + daily_investment

        # BTO leaps
        if leaps_cnt == 0:
            leaps_strike = find_itm_strike(price, leaps_dte, leaps_delta)
            leaps_price = OPT_CONT_SIZE*call_price(price, leaps_strike, rate, leaps_dte/365,vola/100,0)
            leaps_cnt = math.floor(free_capital/leaps_price)
            free_capital = free_capital - leaps_cnt*leaps_price
            leaps_exp_date = date + datetime.timedelta(days=leaps_dte)

        # STC leaps
        elif (leaps_exp_date - date).days <= leaps_dte_limit or idx == (len(series_data)-1):
            free_capital = free_capital + leaps_cnt*OPT_CONT_SIZE*call_price(price, leaps_strike, rate, (leaps_exp_date - date).days/365,mid_iv/100,0)
            leaps_exp_date = None
            leaps_cnt = 0
            leaps_strike = 0

    print(free_capital)
    print(invested)

    return free_capital, invested, len(series_data)

In [93]:
leaps_balance, invested, days = leaps(ticker,leaps_dte,leaps_delta,leaps_dte_limit)

7052830.8920628
71949.99999999802


In [94]:
returns = (leaps_balance/invested) - 1
annual_returns = math.pow((1+returns), 365/days) - 1

print('returns: {}%'.format(round(returns*100,2)))
print('annual returns: {}%'.format(round(annual_returns*100,2)))

returns: 9702.41%
annual returns: 55.75%


# OTM LEAPS

In [95]:
def otm_leaps(ticker,leaps_dte,leaps_delta,leaps_dte_limit):
    leaps_dte = 420
    leaps_delta = 0.4
    leaps_dte_limit = 60

    series_data = get_ticker_data(ticker)

    free_capital = init_cap
    invested = init_cap

    leaps_cnt = 0
    leaps_exp_date = None
    leaps_strike = 0

    for idx, data in enumerate(series_data):
        date, price, divi, vola = data[0].date(), data[1], data[2], data[3]

        # Monthly investment (spread daily)
        free_capital = free_capital + daily_investment
        invested = invested + daily_investment

        # BTO leaps
        if leaps_cnt == 0:
            leaps_strike = find_itm_strike(price, leaps_dte, leaps_delta)
            leaps_price = OPT_CONT_SIZE*call_price(price, leaps_strike, rate, leaps_dte/365,vola/100,0)
            leaps_cnt = math.floor(free_capital/leaps_price)
            free_capital = free_capital - leaps_cnt*leaps_price
            leaps_exp_date = date + datetime.timedelta(days=leaps_dte)

        # STC leaps
        elif (leaps_exp_date - date).days <= leaps_dte_limit or idx == (len(series_data)-1):
            free_capital = free_capital + leaps_cnt*OPT_CONT_SIZE*call_price(price, leaps_strike, rate, (leaps_exp_date - date).days/365,mid_iv/100,0)
            leaps_exp_date = None
            leaps_cnt = 0
            leaps_strike = 0

    print(free_capital)
    print(invested)

    return free_capital, invested, len(series_data)

In [96]:
otm_leaps_balance, invested, days = otm_leaps(ticker,leaps_dte,leaps_delta,leaps_dte_limit)

9612613.891070435
71949.99999999802


In [97]:
returns = (otm_leaps_balance/invested) - 1
annual_returns = math.pow((1+returns), 365/days) - 1

print('returns: {}%'.format(round(returns*100,2)))
print('annual returns: {}%'.format(round(annual_returns*100,2)))

returns: 13260.13%
annual returns: 60.48%
